<a href="https://colab.research.google.com/github/shinjangwoon/jangwoonshin/blob/practice/NCAV(Net_Current_Asset_Value).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NCAV(Net Current Asset Value)

##Basic Setting

In [ ]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN='
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
sample_code = '035720'

In [ ]:
fs = urllib.request.urlopen(FS_URL%sample_code).read()
fs_soup = BeautifulSoup(fs, 'html.parser')

In [ ]:
snap = urllib.request.urlopen(SNAP_URL%sample_code).read()
snap_soup = BeautifulSoup(snap, 'html.parser')

# Market Value

In [ ]:
#type(snap_soup.find('div',{'id':'svdMainGrid1'}))

In [ ]:
#type(snap_soup.find('div',{'id':'svdMainGrid1'}).find_all('td',{'class':'r'}))

In [ ]:
mv_cells = snap_soup.find('div',{'id':'svdMainGrid1'}).find_all('td',{'class':'r'})
market_value = float(mv_cells[8].string.replace(',',''))
market_value

# Current Asset

In [ ]:
ca_cells = fs_soup.find('tr', {'id' : 'p_grid2_2'}).find_all('td', {'class':'r cle'})
current_asset = float(ca_cells[0].string.replace(',',''))
current_asset


# Total Debt

In [ ]:
debt_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('tr', {'class':'rwf rowBold'})
total_debt = float(debt_cells[1].find('td', {'class':'r cle'}).string.replace(',',''))
total_debt

In [ ]:
# 다른 방식
debt_list = []
for debt in debt_cells[1] :
    if debt != '\n':
        debt_list.append(debt.string)

total_debt = float(debt_list[-1].replace(',',''))

total_debt



# Net Income

In [ ]:
income_cells = fs_soup.find('div',{'id': 'divSonikY'}).find_all('tr', {'class':'rwf rowBold'})
net_income = float(income_cells[-1].find_all('td', {'class':'r'})[-3].string.replace(',',''))
net_income

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)
print(company.shape)
company.head(10)

In [ ]:
code_list = company['종목코드']
sample_code_list = code_list[1000:1050]
sample_code_list

In [ ]:
def crawler(li) :
    # url를 세팅한다
    SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN='
    FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

    result = {}

    for code in tqdm(li):
        try:
            fs = urllib.request.urlopen(FS_URL%code).read()
            fs_soup = BeautifulSoup(fs, 'html.parser')

            snap = urllib.request.urlopen(SNAP_URL%code).read()
            snap_soup = BeautifulSoup(snap, 'html.parser')
            # get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]

            # get Market Value
            mv_cells = snap_soup.find('div',{'id':'svdMainGrid1'}).find_all('td',{'class':'r'})
            market_value = float(mv_cells[8].string.replace(',',''))

            # get current asset
            ca_cells = fs_soup.find('tr', {'id' : 'p_grid2_2'}).find_all('td', {'class':'r cle'})
            current_asset = float(ca_cells[0].string.replace(',',''))

            # get total debt
            debt_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('tr', {'class':'rwf rowBold'})
            total_debt = float(debt_cells[1].find('td', {'class':'r cle'}).string.replace(',',''))

            # get net income
            income_cells = fs_soup.find('div',{'id': 'divSonikY'}).find_all('tr', {'class':'rwf rowBold'})
            net_income = float(income_cells[-1].find_all('td', {'class':'r'})[-3].string.replace(',',''))

            # insert into result dict
            result[name] = [code, market_value, current_asset, total_debt, net_income]


        except(ValueError, AttributeError, IndexError, TypeError):
            pass

    # convert dict DataFrame
    result_df = pd.DataFrame(result)

    # transposing DataFrame
    result_df = result_df.transpose()
    
    # net column names
    result_df.columns = ['Code', 'Market_values', 'Current_asset', 'Total_debt', 'Net_income']

    return result_df



In [ ]:
result_df = crawler(sample_code_list)
result_df

In [ ]:
tmp_df = result_df.copy()


In [ ]:
tmp_df['NCAV'] = tmp_df['Current_asset'] - tmp_df['Total_debt']

In [ ]:
tmp_df[(tmp_df['NCAV'] > tmp_df['Market_values'] * 0.3) & (tmp_df['Net_income']>0)]